<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run Images

Goal : have well trained models for image sampling

Data : celeba64,mnist,svhn,celeba32

Models : ConvLNPXL 

Loss : NLLLloss

Runs : 1

In [1]:
import os

os.chdir("..")

In [2]:
import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

In [3]:

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)


In [4]:
args = get_exp_args("exp_qualitative", is_load=False)
len(args)

4

In [5]:
executor=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*24*2,  
                           cpus_per_task=10, 
                           mem='32GB',
                          partition="priority",
                           comment="neurips_appendices",
                            constraint="volta32gb"
                          )

In [6]:
jobs = executor.map_array(Run(), args)

In [7]:
############################################################

In [12]:
jobs

[SlurmJob<job_id=27363068_0, task_id=0, state="RUNNING">,
 SlurmJob<job_id=27363068_1, task_id=0, state="RUNNING">,
 SlurmJob<job_id=27363068_2, task_id=0, state="RUNNING">,
 SlurmJob<job_id=27363068_3, task_id=0, state="RUNNING">]

In [19]:
for job in jobs:
    print("--------------------------------")
    print(job.stdout())

--------------------------------
submitit INFO (2020-06-09 20:44:42,754) - Starting with JobEnvironment(job_id=27363068_0, hostname=learnfair0886, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-06-09 20:44:42,754) - Loading pickle: /private/home/yannd/projects/NPF/logs/27363068_0/27363068_0_submitted.pkl

--- Training mnist/ConvNPFXXL_NllLNPF/run_0 ---

  epoch    train_loss    valid_loss    cp       dur
-------  ------------  ------------  ----  --------
      1    -1142.6384    -1419.8539     +  787.4088
      2    -1560.6912    -1753.5553     +  791.8307
      3    -1690.8613    -1475.3137        789.1432
      4    -1763.1967    -1898.2776     +  788.5574
      5    -1828.5915    -1913.4310     +  790.2562
      6    -1865.1989    -1907.3858        797.7764
      7    -1888.7882    -2009.6174     +  790.1354
      8    -1911.3203    -1803.7968        788.6665
      9    -1921.4671    -1983.2658        788.9310
     10    -1950.2479    -1961.9771        792.4412
 

In [54]:
for j in jobs_32:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
submitit ERROR (2020-05-24 20:41:35,817) - Submitted job triggered an exception
ERROR:submitit:Submitted job triggered an exception
Traceback (most recent call last):
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/_submit.py", line 6, in <module>
    submitit_main()
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 186, in submitit_main
    process_job(args.folder)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 179, in process_job
    raise error
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submi

In [13]:
for job in jobs:
    job.cancel()